In [51]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [52]:
import itertools
import os
import math
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
layers = keras.layers

# This code was tested with TensorFlow v1.7
print("You have TensorFlow version", tf.__version__)

You have TensorFlow version 1.7.0


In [53]:
URL = "https://goo.gl/4TUz1D"
path = tf.keras.utils.get_file(URL.split('/')[-1], URL)

In [54]:
# Convert the data to a Pandas data frame
data = pd.read_csv(path)

In [55]:
# Shuffle the data
data = data.sample(frac=1)

# Print the first 5 rows
data.head()

,Brand,Model/details,Memory,Color,Price,Description,Link,I don't know,Date posted,Date scraped
1046,iPhone,SE,32,guld,1450,"\n \n iPhone SE, 32 GB, guld...",https://www.dba.dk/iphone-se-32-gb-guld/id-105...,1052149915,27. januar kl. 13.12,30. januar 2019 11:19
358,OnePlus,6,128,NaN,3200,"\n \n OnePlus 6, 128 , Perfe...",https://www.dba.dk/oneplus-6-128-perfekt-h/id-...,1052219546,29. januar 2019 kl. 16.35,30. januar 2019 11:16
1139,Sony,Sony Xperia Z5 32GB Sort,Sony Xperia Z5 32GB Sort,sort,1499,\n \n Sony Sony Xperia Z5 32...,https://www.dba.dk/sony-sony-xperia-z5-32gb-so...,108022964,27. januar kl. 6.07,30. januar 2019 12:01
2075,Samsung,S6 edge,32,NaN,800,"\n \n Samsung S6 edge, 32 , ...",https://www.dba.dk/samsung-s6-edge-32-rimel/id...,1052001709,22. januar kl. 15.52,30. januar 2019 12:01
792,iPhone,8,64,hvid,4000,"\n \n iPhone 8, 64 GB, hvid,...",https://www.dba.dk/iphone-8/id-1052184663/,1052184663,28. januar kl. 12.37,30. januar 2019 11:12


In [56]:
# Do some preprocessing to limit the # of wine varities in the dataset
# data = data[pd.notnull(data['country'])]
#data = data[pd.notnull(data['Price'])]
data = data[pd.notnull(data['Description'])]
data = data[pd.notnull(data['Model/details'])]


# data = data.drop(data.columns[0], axis=1) 



In [57]:
brand_threshold = 400 # Anything that occurs less than this will be removed.
value_counts = data['Brand'].value_counts()
value_counts.head()
to_remove = value_counts[value_counts <= brand_threshold].index
data.replace(to_remove, np.nan, inplace=True)
data = data[pd.notnull(data['Brand'])]


In [58]:
brand_threshold = 10 # Anything that occurs less than this will be removed.
value_counts = data['Model/details'].value_counts()
value_counts.head()
to_remove = value_counts[value_counts <= brand_threshold].index
data.replace(to_remove, np.nan, inplace=True)
data = data[pd.notnull(data['Model/details'])]

In [59]:
data['Model/details'].value_counts()

6S         213
7          199
6          179
X          106
5S          97
8           96
SE          95
7 Plus      65
8 Plus      60
XS Max      49
6 Plus      38
XS          33
6S Plus     33
5           33
5C          11
Name: Model/details, dtype: int64

In [60]:
# Split data into train and test
train_size = int(len(data) * .9)
print ("Train size: %d" % train_size)
print ("Test size: %d" % (len(data) - train_size))

Train size: 1176
Test size: 131


In [61]:
# Train features
description_train = data['Description'][:train_size]
brand_train = data['Model/details'][:train_size]

# Train labels
labels_train = data['Price'][:train_size]

# Test features
description_test = data['Description'][train_size:]
brand_test = data['Model/details'][train_size:]

# Test labels
labels_test = data['Price'][train_size:]

In [62]:
# Create a tokenizer to preprocess our text descriptions
vocab_size = 100 # This is a hyperparameter, experiment with different values for your dataset
tokenize = keras.preprocessing.text.Tokenizer(num_words=vocab_size, char_level=False)
tokenize.fit_on_texts(description_train) # only fit on train



In [63]:
import pickle 

# py_save_object(tokenize, "my_tokenizer_2.p", pickle = "pickle")

pickle_out = open("tokenizer1.obj", "wb")
pickle.dump(tokenize, pickle_out)
pickle_out.close()

In [64]:
# Wide feature 1: sparse bag of words (bow) vocab_size vector 
description_bow_train = tokenize.texts_to_matrix(description_train)
description_bow_test = tokenize.texts_to_matrix(description_test)

In [65]:
# Wide feature 2: one-hot vector of variety categories

# Use sklearn utility to convert label strings to numbered index
encoder = LabelEncoder()
encoder.fit(brand_train)
brand_train = encoder.transform(brand_train)
brand_test = encoder.transform(brand_test)
num_classes = np.max(brand_train) + 1

# Convert labels to one hot
brand_train = keras.utils.to_categorical(brand_train, num_classes)
brand_test = keras.utils.to_categorical(brand_test, num_classes)

In [66]:
pickle_out2 = open("encoder1.p", 'wb')
pickle.dump(encoder, pickle_out2)
pickle_out2.close()

In [67]:
# Define our wide model with the functional API
bow_inputs = layers.Input(shape=(vocab_size,))
brand_inputs = layers.Input(shape=(num_classes,))
merged_layer = layers.concatenate([bow_inputs, brand_inputs])
merged_layer = layers.Dense(256, activation='relu')(merged_layer)
predictions = layers.Dense(1)(merged_layer)
wide_model = keras.Model(inputs=[bow_inputs, brand_inputs], outputs=predictions)

In [68]:
wide_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
print(wide_model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 15)           0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 115)          0           input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 256)          29696       concatenate_1[0][0]              
__________

In [69]:
# Deep model feature: word embeddings of wine descriptions
train_embed = tokenize.texts_to_sequences(description_train)
test_embed = tokenize.texts_to_sequences(description_test)

max_seq_length = 170
train_embed = keras.preprocessing.sequence.pad_sequences(
    train_embed, maxlen=max_seq_length, padding="post")
test_embed = keras.preprocessing.sequence.pad_sequences(
    test_embed, maxlen=max_seq_length, padding="post")

In [70]:
# Define our deep model with the Functional API
deep_inputs = layers.Input(shape=(max_seq_length,))
embedding = layers.Embedding(vocab_size, 8, input_length=max_seq_length)(deep_inputs)
embedding = layers.Flatten()(embedding)
embed_out = layers.Dense(1)(embedding)
deep_model = keras.Model(inputs=deep_inputs, outputs=embed_out)
print(deep_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 170)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 170, 8)            800       
_________________________________________________________________
flatten_1 (Flatten)          (None, 1360)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 1361      
Total params: 2,161
Trainable params: 2,161
Non-trainable params: 0
_________________________________________________________________
None


In [71]:
deep_model.compile(loss='mse',
                       optimizer='adam',
                       metrics=['accuracy'])

In [72]:
# Combine wide and deep into one model
merged_out = layers.concatenate([wide_model.output, deep_model.output])
merged_out = layers.Dense(1)(merged_out)
combined_model = keras.Model(wide_model.input + [deep_model.input], merged_out)
print(combined_model.summary())

combined_model.compile(loss='mse',
                       optimizer='adam',
                       metrics=['accuracy'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 15)           0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 170)          0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 115)          0           input_1[0][0]                    
                                                                 input_2[0][0]                    
__________

In [73]:
# Run training
combined_model.fit([description_bow_train, brand_train] + [train_embed], labels_train, epochs=7000, batch_size=128)

Epoch 1/7000
1176/1176 [==============================] - 0s 272us/step - loss: 13446151.3605 - acc: 0.0000e+00
Epoch 2/7000
1176/1176 [==============================] - 0s 33us/step - loss: 13390078.7755 - acc: 0.0000e+00
Epoch 3/7000
1176/1176 [==============================] - 0s 34us/step - loss: 13328625.0612 - acc: 0.0000e+00
Epoch 4/7000
1176/1176 [==============================] - 0s 36us/step - loss: 13261426.6939 - acc: 0.0000e+00
Epoch 5/7000
1176/1176 [==============================] - 0s 34us/step - loss: 13187533.0646 - acc: 0.0000e+00
Epoch 6/7000
1176/1176 [==============================] - 0s 36us/step - loss: 13105444.1565 - acc: 0.0000e+00
Epoch 7/7000
1176/1176 [==============================] - 0s 32us/step - loss: 13014291.4830 - acc: 0.0000e+00
Epoch 8/7000
1176/1176 [==============================] - 0s 37us/step - loss: 12913657.2109 - acc: 0.0000e+00
Epoch 9/7000
1176/1176 [==============================] - 0s 34us/step - loss: 12798215.7823 - acc: 0.0000e+00


1176/1176 [==============================] - 0s 39us/step - loss: 5150928.4983 - acc: 0.0000e+00
Epoch 75/7000
1176/1176 [==============================] - 0s 34us/step - loss: 5114375.3878 - acc: 0.0000e+00
Epoch 76/7000
1176/1176 [==============================] - 0s 39us/step - loss: 5078096.2372 - acc: 0.0000e+00
Epoch 77/7000
1176/1176 [==============================] - 0s 35us/step - loss: 5039745.5408 - acc: 0.0000e+00
Epoch 78/7000
1176/1176 [==============================] - 0s 33us/step - loss: 5001846.1199 - acc: 8.5034e-04
Epoch 79/7000
1176/1176 [==============================] - 0s 34us/step - loss: 4965083.7228 - acc: 0.0000e+00
Epoch 80/7000
1176/1176 [==============================] - 0s 32us/step - loss: 4926972.4813 - acc: 8.5034e-04
Epoch 81/7000
1176/1176 [==============================] - 0s 36us/step - loss: 4888022.3724 - acc: 0.0000e+00
Epoch 82/7000
1176/1176 [==============================] - 0s 35us/step - loss: 4846506.9133 - acc: 0.0000e+00
Epoch 83/7000
1

1176/1176 [==============================] - 0s 31us/step - loss: 3024084.5119 - acc: 0.0000e+00
Epoch 148/7000
1176/1176 [==============================] - 0s 34us/step - loss: 3014598.8784 - acc: 0.0000e+00
Epoch 149/7000
1176/1176 [==============================] - 0s 31us/step - loss: 3004634.2800 - acc: 0.0000e+00
Epoch 150/7000
1176/1176 [==============================] - 0s 33us/step - loss: 2996984.7891 - acc: 0.0017
Epoch 151/7000
1176/1176 [==============================] - 0s 33us/step - loss: 2988185.8635 - acc: 8.5034e-04
Epoch 152/7000
1176/1176 [==============================] - 0s 33us/step - loss: 2978254.3146 - acc: 0.0017
Epoch 153/7000
1176/1176 [==============================] - 0s 32us/step - loss: 2970539.8690 - acc: 8.5034e-04
Epoch 154/7000
1176/1176 [==============================] - 0s 34us/step - loss: 2961892.1186 - acc: 8.5034e-04
Epoch 155/7000
1176/1176 [==============================] - 0s 31us/step - loss: 2953443.3410 - acc: 0.0000e+00
Epoch 156/7000


Epoch 221/7000
1176/1176 [==============================] - 0s 31us/step - loss: 2573872.2730 - acc: 0.0017
Epoch 222/7000
1176/1176 [==============================] - 0s 32us/step - loss: 2569520.3540 - acc: 8.5034e-04
Epoch 223/7000
1176/1176 [==============================] - 0s 32us/step - loss: 2567251.6361 - acc: 0.0000e+00
Epoch 224/7000
1176/1176 [==============================] - 0s 31us/step - loss: 2561494.0068 - acc: 0.0026
Epoch 225/7000
1176/1176 [==============================] - 0s 32us/step - loss: 2556594.5270 - acc: 0.0000e+00
Epoch 226/7000
1176/1176 [==============================] - 0s 32us/step - loss: 2554200.6716 - acc: 8.5034e-04
Epoch 227/7000
1176/1176 [==============================] - 0s 31us/step - loss: 2548276.7623 - acc: 0.0000e+00
Epoch 228/7000
1176/1176 [==============================] - 0s 33us/step - loss: 2543810.2007 - acc: 0.0000e+00
Epoch 229/7000
1176/1176 [==============================] - 0s 33us/step - loss: 2539695.7177 - acc: 8.5034e-04


1176/1176 [==============================] - 0s 31us/step - loss: 2286787.5200 - acc: 0.0000e+00
Epoch 295/7000
1176/1176 [==============================] - 0s 32us/step - loss: 2283094.2330 - acc: 0.0017
Epoch 296/7000
1176/1176 [==============================] - 0s 31us/step - loss: 2279877.5544 - acc: 0.0000e+00
Epoch 297/7000
1176/1176 [==============================] - 0s 32us/step - loss: 2275848.0347 - acc: 0.0000e+00
Epoch 298/7000
1176/1176 [==============================] - 0s 34us/step - loss: 2272953.9762 - acc: 8.5034e-04
Epoch 299/7000
1176/1176 [==============================] - 0s 32us/step - loss: 2270633.7585 - acc: 8.5034e-04
Epoch 300/7000
1176/1176 [==============================] - 0s 32us/step - loss: 2266604.0315 - acc: 0.0000e+00
Epoch 301/7000
1176/1176 [==============================] - 0s 34us/step - loss: 2263398.0616 - acc: 0.0017
Epoch 302/7000
1176/1176 [==============================] - 0s 33us/step - loss: 2260340.8571 - acc: 0.0017
Epoch 303/7000
1176

...

1176/1176 [==============================] - 0s 37us/step - loss: 8050.5387 - acc: 0.0170
Epoch 6662/7000
1176/1176 [==============================] - 0s 34us/step - loss: 8101.6590 - acc: 0.0077
Epoch 6663/7000
1176/1176 [==============================] - 0s 38us/step - loss: 7863.6943 - acc: 0.0094
Epoch 6664/7000
1176/1176 [==============================] - 0s 40us/step - loss: 8426.2847 - acc: 0.0128
Epoch 6665/7000
1176/1176 [==============================] - 0s 34us/step - loss: 9202.6127 - acc: 0.0051
Epoch 6666/7000
1176/1176 [==============================] - 0s 36us/step - loss: 9093.7756 - acc: 0.0068
Epoch 6667/7000
1176/1176 [==============================] - 0s 36us/step - loss: 9388.4732 - acc: 0.0068
Epoch 6668/7000
1176/1176 [==============================] - 0s 33us/step - loss: 8486.3132 - acc: 0.0060
Epoch 6669/7000
1176/1176 [==============================] - 0s 34us/step - loss: 8170.4088 - acc: 0.0170
Epoch 6670/7000
1176/1176 [==============================] - 0

1176/1176 [==============================] - 0s 33us/step - loss: 7716.7435 - acc: 0.0145
Epoch 6739/7000
1176/1176 [==============================] - 0s 39us/step - loss: 7453.4724 - acc: 0.0170
Epoch 6740/7000
1176/1176 [==============================] - 0s 35us/step - loss: 8051.1575 - acc: 0.0136
Epoch 6741/7000
1176/1176 [==============================] - 0s 33us/step - loss: 7954.0861 - acc: 0.0060
Epoch 6742/7000
1176/1176 [==============================] - 0s 36us/step - loss: 8019.6096 - acc: 0.0145
Epoch 6743/7000
1176/1176 [==============================] - 0s 33us/step - loss: 7875.4545 - acc: 0.0145
Epoch 6744/7000
1176/1176 [==============================] - 0s 35us/step - loss: 7672.4823 - acc: 0.0094
Epoch 6745/7000
1176/1176 [==============================] - 0s 32us/step - loss: 8225.0242 - acc: 0.0119
Epoch 6746/7000
1176/1176 [==============================] - 0s 35us/step - loss: 9464.4915 - acc: 0.0034
Epoch 6747/7000
1176/1176 [==============================] - 0

1176/1176 [==============================] - 0s 39us/step - loss: 7619.7009 - acc: 0.0145
Epoch 6816/7000
1176/1176 [==============================] - 0s 35us/step - loss: 7576.9002 - acc: 0.0187
Epoch 6817/7000
1176/1176 [==============================] - 0s 36us/step - loss: 7556.3832 - acc: 0.0136
Epoch 6818/7000
1176/1176 [==============================] - 0s 32us/step - loss: 7294.7887 - acc: 0.0145
Epoch 6819/7000
1176/1176 [==============================] - 0s 35us/step - loss: 7692.6053 - acc: 0.0136
Epoch 6820/7000
1176/1176 [==============================] - 0s 33us/step - loss: 7531.4813 - acc: 0.0179
Epoch 6821/7000
1176/1176 [==============================] - 0s 34us/step - loss: 7571.8437 - acc: 0.0162
Epoch 6822/7000
1176/1176 [==============================] - 0s 32us/step - loss: 7303.2963 - acc: 0.0153
Epoch 6823/7000
1176/1176 [==============================] - 0s 35us/step - loss: 8370.5775 - acc: 0.0051
Epoch 6824/7000
1176/1176 [==============================] - 0

1176/1176 [==============================] - 0s 35us/step - loss: 7551.3832 - acc: 0.0136
Epoch 6892/7000
1176/1176 [==============================] - 0s 40us/step - loss: 7303.8303 - acc: 0.0170
Epoch 6893/7000
1176/1176 [==============================] - 0s 38us/step - loss: 7687.7996 - acc: 0.0179
Epoch 6894/7000
1176/1176 [==============================] - 0s 34us/step - loss: 7269.7928 - acc: 0.0077
Epoch 6895/7000
1176/1176 [==============================] - 0s 34us/step - loss: 7132.3435 - acc: 0.0162
Epoch 6896/7000
1176/1176 [==============================] - 0s 38us/step - loss: 7106.4235 - acc: 0.0187
Epoch 6897/7000
1176/1176 [==============================] - 0s 37us/step - loss: 7323.0228 - acc: 0.0128
Epoch 6898/7000
1176/1176 [==============================] - 0s 35us/step - loss: 7388.9259 - acc: 0.0068
Epoch 6899/7000
1176/1176 [==============================] - 0s 38us/step - loss: 7345.3614 - acc: 0.0153
Epoch 6900/7000
1176/1176 [==============================] - 0

1176/1176 [==============================] - 0s 36us/step - loss: 6838.7046 - acc: 0.0221
Epoch 6969/7000
1176/1176 [==============================] - 0s 41us/step - loss: 6967.7890 - acc: 0.0153
Epoch 6970/7000
1176/1176 [==============================] - 0s 34us/step - loss: 6856.0909 - acc: 0.0170
Epoch 6971/7000
1176/1176 [==============================] - 0s 36us/step - loss: 6868.0890 - acc: 0.0187
Epoch 6972/7000
1176/1176 [==============================] - 0s 33us/step - loss: 6775.2737 - acc: 0.0204
Epoch 6973/7000
1176/1176 [==============================] - 0s 36us/step - loss: 6979.0554 - acc: 0.0153
Epoch 6974/7000
1176/1176 [==============================] - 0s 34us/step - loss: 6874.7630 - acc: 0.0136
Epoch 6975/7000
1176/1176 [==============================] - 0s 35us/step - loss: 6813.0953 - acc: 0.0196
Epoch 6976/7000
1176/1176 [==============================] - 0s 33us/step - loss: 6874.7802 - acc: 0.0162
Epoch 6977/7000
1176/1176 [==============================] - 0

In [74]:
combined_model.evaluate([description_bow_test, brand_test] + [test_embed], labels_test, batch_size=128)

131/131 [==============================] - 0s 388us/step


[3884809.4914122135, 0.0]

In [75]:
combined_model.save('WideandDeep_model_1.h5')


In [76]:
predictions = combined_model.predict([description_bow_test, brand_test] + [test_embed])


In [77]:
# Compare predictions with actual values for the first few items in our test dataset
num_predictions = 40
diff = 0

for i in range(num_predictions):
    val = predictions[i]
    print(description_test.iloc[i])
    print('Predicted: ', val[0], 'Actual: ', labels_test.iloc[i], '\n')
    diff += abs(val[0] - labels_test.iloc[i])


        
            iPhone 7, 32 GB, sort, God
        
    Jeg sælger min iPhone, og smider en oplader oven i prisen. En iPhone med små synlige skader i kanterne. En lille ridse øverst i højre hjørne, og en lille nederst i venstre hjørne
Predicted:  1353.6643 Actual:  2000 


        
            iPhone 6 Plus, 16 GB, hvid, God
        
    Sælger min iPhone 6 plus grundet skift til nyere model.Virker som den skal. ca 3-4 år gammel.Lader medfølger ikke da den er gået i stykker. Kasse medfølger. 
Predicted:  707.3948 Actual:  1300 


        
            iPhone 6S, 64 GB, hvid, God
        
    Virker som den skal har riser skærm og bagcover 
Predicted:  2193.8933 Actual:  1500 


        
            iPhone 5C, 16 GB, blå, Perfekt
        
    Helt ny skærm. Virker 100 %. 
Predicted:  888.3342 Actual:  500 


        
            iPhone 7, 128 GB, God
        
    Sælger min iPhone 7 rosegold da jeg har fået en ny telefon. Der er for nyligt skiftet højtaler og skærm på den, kvitteri

In [78]:
# Compare the average difference between actual price and the model's predicted price
print('Average prediction difference: ', diff / num_predictions)

Average prediction difference:  785.885173034668
